In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
input_data='./input_data/'
output_data='./output_data/'


In [3]:
daily_raw = pd.read_csv(input_data + "chavuma.csv")
daily=daily_raw.dropna().copy()
daily['Date']=pd.to_datetime(daily['Date'],format="%d/%m/%Y")
daily=daily.set_index(pd.DatetimeIndex(daily['Date']))
daily['MonthId']=daily['Year']+daily['Month']/100
daily['WaterMonth']=daily.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)
daily

,Date,Month,Day,Year,WaterYear,Flow,MonthId,WaterMonth
Date,,,,,,,,
1959-10-10,1959-10-10,10,10,1959,1959,49.0,1959.10,1
1959-10-11,1959-10-11,10,11,1959,1959,49.0,1959.10,1
1959-10-12,1959-10-12,10,12,1959,1959,49.0,1959.10,1
1959-10-13,1959-10-13,10,13,1959,1959,49.0,1959.10,1
1959-10-14,1959-10-14,10,14,1959,1959,47.0,1959.10,1
...,...,...,...,...,...,...,...,...
2020-06-16,2020-06-16,6,16,2020,2019,480.0,2020.06,9
2020-06-17,2020-06-17,6,17,2020,2019,470.0,2020.06,9
2020-06-18,2020-06-18,6,18,2020,2019,461.0,2020.06,9


In [4]:
daily['GapEnd']=daily['Date']!=daily['Date'].shift(1)+pd.DateOffset(1)
daily['GapStart']=np.where(daily['GapEnd']==True,daily['Date'].shift(1),pd.NaT)
daily=daily.astype({'GapStart': 'datetime64'})
daily

,Date,Month,Day,Year,WaterYear,Flow,MonthId,WaterMonth,GapEnd,GapStart
Date,,,,,,,,,,
1959-10-10,1959-10-10,10,10,1959,1959,49.0,1959.10,1,True,NaT
1959-10-11,1959-10-11,10,11,1959,1959,49.0,1959.10,1,False,NaT
1959-10-12,1959-10-12,10,12,1959,1959,49.0,1959.10,1,False,NaT
1959-10-13,1959-10-13,10,13,1959,1959,49.0,1959.10,1,False,NaT
1959-10-14,1959-10-14,10,14,1959,1959,47.0,1959.10,1,False,NaT
...,...,...,...,...,...,...,...,...,...,...
2020-06-16,2020-06-16,6,16,2020,2019,480.0,2020.06,9,False,NaT
2020-06-17,2020-06-17,6,17,2020,2019,470.0,2020.06,9,False,NaT
2020-06-18,2020-06-18,6,18,2020,2019,461.0,2020.06,9,False,NaT


In [5]:
gaps=daily.loc[daily['GapEnd']==True][['GapStart','Flow']].reset_index().rename(columns={'Date':'GapEnd','Flow':'FlowEnd'})
gaps=gaps.drop(0)
gaps.head(2)

,GapEnd,GapStart,FlowEnd
1,1960-04-01,1960-03-28,5260.0
2,1960-06-25,1960-05-07,299.0


In [6]:
gaps['FlowStart']=gaps.join(daily, on='GapStart',lsuffix='_L',how='inner')['Flow']
gaps['Days']=((gaps['GapEnd']-pd.DateOffset(1))-gaps['GapStart']).dt.days

In [7]:
gaps.head(2)

,GapEnd,GapStart,FlowEnd,FlowStart,Days
1,1960-04-01,1960-03-28,5260.0,5477.0,3
2,1960-06-25,1960-05-07,299.0,1004.0,48


In [8]:
fills=pd.DataFrame()

for row in gaps.itertuples():
    fillset=pd.DataFrame({'GapNo': row.Index, 'Date':pd.date_range(start=row.GapStart+pd.DateOffset(1), end=row.GapEnd-pd.DateOffset(1))})
    joined=fillset.join(gaps,on='GapNo',lsuffix='_L')
    fillset['DayNo']=(fillset['Date']-joined['GapStart']).dt.days
    fillset['Flow']=joined['FlowStart']+((joined['FlowEnd']-joined['FlowStart'])/(joined['Days']+1)*fillset['DayNo'])
    fills=fills.append(fillset)

fills=fills.set_index('Date')
fills

,GapNo,DayNo,Flow
Date,,,
1960-03-29,1,1,5422.750000
1960-03-30,1,2,5368.500000
1960-03-31,1,3,5314.250000
1960-05-08,2,1,989.612245
1960-05-09,2,2,975.224490
...,...,...,...
2000-07-27,41,27,109.937500
2000-07-28,41,28,107.750000
2000-07-29,41,29,105.562500


In [9]:
fills['Year']=fills.index.year
fills['Month']=fills.index.month
fills['Day']=fills.index.day
fills['MonthId']=fills['Year']+fills['Month']/100
fills['WaterYear']=fills.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
fills=fills.drop(['GapNo','DayNo'],axis=1)
daily=daily.drop(['GapStart','GapEnd'],axis=1)
fills

,Flow,Year,Month,Day,MonthId,WaterYear
Date,,,,,,
1960-03-29,5422.750000,1960,3,29,1960.03,1959.0
1960-03-30,5368.500000,1960,3,30,1960.03,1959.0
1960-03-31,5314.250000,1960,3,31,1960.03,1959.0
1960-05-08,989.612245,1960,5,8,1960.05,1959.0
1960-05-09,975.224490,1960,5,9,1960.05,1959.0
...,...,...,...,...,...,...
2000-07-27,109.937500,2000,7,27,2000.07,1999.0
2000-07-28,107.750000,2000,7,28,2000.07,1999.0
2000-07-29,105.562500,2000,7,29,2000.07,1999.0


In [10]:
daily=daily.append(fills)
daily

,Date,Month,Day,Year,WaterYear,Flow,MonthId,WaterMonth
Date,,,,,,,,
1959-10-10,1959-10-10,10,10,1959,1959.0,49.0000,1959.10,1.0
1959-10-11,1959-10-11,10,11,1959,1959.0,49.0000,1959.10,1.0
1959-10-12,1959-10-12,10,12,1959,1959.0,49.0000,1959.10,1.0
1959-10-13,1959-10-13,10,13,1959,1959.0,49.0000,1959.10,1.0
1959-10-14,1959-10-14,10,14,1959,1959.0,47.0000,1959.10,1.0
...,...,...,...,...,...,...,...,...
2000-07-27,NaT,7,27,2000,1999.0,109.9375,2000.07,NaN
2000-07-28,NaT,7,28,2000,1999.0,107.7500,2000.07,NaN
2000-07-29,NaT,7,29,2000,1999.0,105.5625,2000.07,NaN


In [11]:
monthly=daily.groupby(['MonthId','Year','Month']).size().to_frame(name="DaysRecorded").reset_index(['Month','Year'])
monthly['Days']=monthly.apply(lambda x: datetime.datetime(x['Year'],x['Month'],1),axis=1).dt.daysinmonth
monthly

,Year,Month,DaysRecorded,Days
MonthId,,,,
1959.10,1959,10,22,31
1959.11,1959,11,30,30
1959.12,1959,12,31,31
1960.01,1960,1,31,31
1960.02,1960,2,29,29
...,...,...,...,...
2020.02,2020,2,29,29
2020.03,2020,3,31,31
2020.04,2020,4,30,30


In [12]:
monthly.drop(monthly.loc[monthly['DaysRecorded']!=monthly['Days']].index)
monthly=monthly.drop(['DaysRecorded'],axis=1)
monthly['Day']=1
monthly['DateStart']=pd.to_datetime(monthly[['Year','Month','Day']])
monthly=monthly.drop('Day',1)


monthly['WaterYear']=monthly.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
monthly['WaterMonth']=monthly.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)


monthly

,Year,Month,Days,DateStart,WaterYear,WaterMonth
MonthId,,,,,,
1959.10,1959,10,31,1959-10-01,1959,1
1959.11,1959,11,30,1959-11-01,1959,2
1959.12,1959,12,31,1959-12-01,1959,3
1960.01,1960,1,31,1960-01-01,1959,4
1960.02,1960,2,29,1960-02-01,1959,5
...,...,...,...,...,...,...
2020.02,2020,2,29,2020-02-01,2019,5
2020.03,2020,3,31,2020-03-01,2019,6
2020.04,2020,4,30,2020-04-01,2019,7


In [13]:
monthly['Flow_min']=daily[['MonthId','Flow']].groupby('MonthId').min()
monthly['Flow_mean']=daily[['MonthId','Flow']].groupby('MonthId').mean()
monthly['Flow_median']=daily[['MonthId','Flow']].groupby('MonthId').median()
monthly['Flow_max']=daily[['MonthId','Flow']].groupby('MonthId').max()
monthly['Flow_range']=monthly['Flow_max']-monthly['Flow_min']
monthly[['Flow_min','Flow_mean','Flow_median','Flow_max','Flow_range']]
monthly

,Year,Month,Days,DateStart,WaterYear,WaterMonth,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_range
MonthId,,,,,,,,,,,
1959.10,1959,10,31,1959-10-01,1959,1,38.0,42.545455,43.0,49.0,11.0
1959.11,1959,11,30,1959-11-01,1959,2,43.0,64.300000,70.0,83.0,40.0
1959.12,1959,12,31,1959-12-01,1959,3,86.0,128.612903,124.0,193.0,107.0
1960.01,1960,1,31,1960-01-01,1959,4,204.0,368.838710,419.0,453.0,249.0
1960.02,1960,2,29,1960-02-01,1959,5,427.0,1013.310345,930.0,2014.0,1587.0
...,...,...,...,...,...,...,...,...,...,...,...
2020.02,2020,2,29,2020-02-01,2019,5,1163.0,3096.862069,2931.0,5006.0,3843.0
2020.03,2020,3,31,2020-03-01,2019,6,2838.0,3925.354839,3952.0,5290.0,2452.0
2020.04,2020,4,30,2020-04-01,2019,7,1994.0,4253.533333,4433.5,5825.0,3831.0


In [14]:
yearly=monthly[['WaterYear']].groupby('WaterYear').count()

yearly['Flow_min']=daily[['WaterYear','Flow']].groupby('WaterYear').min()
yearly['Flow_median']=daily[['WaterYear','Flow']].groupby('WaterYear').median()
yearly['Flow_mean']=daily[['WaterYear','Flow']].groupby('WaterYear').mean()
yearly['Flow_max']=daily[['WaterYear','Flow']].groupby('WaterYear').max()
yearly['Flow_range']=yearly['Flow_max']-yearly['Flow_min']


yearly

,Flow_min,Flow_median,Flow_mean,Flow_max,Flow_range
WaterYear,,,,,
1959,38.000000,262.000000,814.530812,5477.000000,5439.000000
1960,55.000000,287.000000,1048.950685,5480.000000,5425.000000
1961,77.000000,432.000000,1123.441096,5725.000000,5648.000000
1962,87.000000,392.000000,1188.843836,5469.000000,5382.000000
1963,79.000000,299.500000,535.295082,1955.000000,1876.000000
...,...,...,...,...,...
2015,50.894147,237.760565,799.260528,3429.601153,3378.707006
2016,48.199892,181.728401,508.636628,3335.639173,3287.439281
2017,43.368379,404.652697,986.491220,4916.311666,4872.943287


In [15]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly['Flow_min']=daily[['WaterMonth','Flow']].groupby('WaterMonth').min()
calmonthly['Flow_mean']=daily[['WaterMonth','Flow']].groupby('WaterMonth').mean()
calmonthly['Flow_median']=daily[['WaterMonth','Flow']].groupby('WaterMonth').median()
calmonthly['Flow_max']=daily[['WaterMonth','Flow']].groupby('WaterMonth').max()
calmonthly['Flow_std']=daily[['WaterMonth','Flow']].groupby('WaterMonth').std()
calmonthly['Flow_coefvar']=(calmonthly['Flow_std']/calmonthly['Flow_mean']*100).round(1)


calmonthly

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar
WaterMonth,,,,,,,,
1,Oct,10,24.0,65.951437,63.000000,167.000000,23.045254,34.9
2,Nov,11,28.0,87.681907,79.000000,290.000000,41.378963,47.2
3,Dec,12,45.0,228.955284,169.962293,2677.000000,225.119560,98.3
4,Jan,1,93.0,704.759017,483.982924,5827.000000,702.256724,99.6
5,Feb,2,147.0,1456.075928,1157.280278,6083.000000,1082.587883,74.3
6,Mar,3,356.0,2084.532872,1791.693068,7139.000000,1214.300542,58.3
7,Apr,4,391.0,1916.916348,1462.000000,7416.000000,1215.164928,63.4
8,May,5,171.0,720.852008,641.929403,3020.568902,375.131541,52.0
9,Jun,6,108.0,320.948853,308.000000,863.000000,113.127873,35.2


In [16]:
calmonthly['Flow_P95']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.05)
calmonthly['Flow_P90']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.1)
#calmonthly['Flow_P80']=monthly[['Month','Flow_mean']].groupby('Month').quantile(0.2)
calmonthly['Flow_P75']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.25)
calmonthly['Flow_P50']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.5)
calmonthly['Flow_P25']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.75)
#calmonthly['Flow_P20']=monthly[['Month','Flow_mean']].groupby('Month').quantile(0.8)
calmonthly['Flow_P10']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.9)
calmonthly['Flow_P05']=monthly[['WaterMonth','Flow_mean']].groupby('WaterMonth').quantile(0.95)

calmonthly

,MonthName,Month,Flow_min,Flow_mean,Flow_median,Flow_max,Flow_std,Flow_coefvar,Flow_P95,Flow_P90,Flow_P75,Flow_P50,Flow_P25,Flow_P10,Flow_P05
WaterMonth,,,,,,,,,,,,,,,
1,Oct,10,24.0,65.951437,63.000000,167.000000,23.045254,34.9,30.677419,40.836143,51.290323,63.500000,77.177111,92.612903,96.290323
2,Nov,11,28.0,87.681907,79.000000,290.000000,41.378963,47.2,45.300000,52.093964,61.000000,79.700000,104.154406,122.733333,154.400000
3,Dec,12,45.0,228.955284,169.962293,2677.000000,225.119560,98.3,87.290323,97.709677,132.308482,175.774194,278.000000,391.193548,482.365591
4,Jan,1,93.0,704.759017,483.982924,5827.000000,702.256724,99.6,181.580645,252.838710,338.645161,521.354839,762.032258,1359.129032,1858.741935
5,Feb,2,147.0,1456.075928,1157.280278,6083.000000,1082.587883,74.3,442.250000,590.821429,749.857143,1246.464286,1825.075939,2848.321429,3452.964286
6,Mar,3,356.0,2084.532872,1791.693068,7139.000000,1214.300542,58.3,721.593564,731.119812,1325.000000,1886.935484,2688.483871,4043.370968,4336.000000
7,Apr,4,391.0,1916.916348,1462.000000,7416.000000,1215.164928,63.4,630.833333,746.600000,1124.233333,1580.566667,2481.966667,3156.533333,4080.036691
8,May,5,171.0,720.852008,641.929403,3020.568902,375.131541,52.0,322.741935,431.612903,530.838710,680.483871,850.924018,1100.185398,1260.354839
9,Jun,6,108.0,320.948853,308.000000,863.000000,113.127873,35.2,168.833333,188.166667,265.200000,321.866667,389.166667,489.766667,508.243411


In [17]:
daily.to_csv(output_data + 'chavuma_daily.csv')
monthly.to_csv(output_data + 'chavuma_monthly.csv')
yearly.to_csv(output_data + 'chavuma_yearly.csv')
calmonthly.to_csv(output_data + 'chavuma_calmonthly.csv')

In [18]:
daily['WaterMonth']=daily.apply((lambda x: (x['Month']-9 if x['Month']>=10 else x['Month']+3)),axis=1)
daily=daily.drop('Date',1)
daily=daily.reset_index().sort_values('Date').set_index('Date')
daily

,Month,Day,Year,WaterYear,Flow,MonthId,WaterMonth
Date,,,,,,,
1959-10-10,10,10,1959,1959.0,49.0000,1959.10,1
1959-10-11,10,11,1959,1959.0,49.0000,1959.10,1
1959-10-12,10,12,1959,1959.0,49.0000,1959.10,1
1959-10-13,10,13,1959,1959.0,49.0000,1959.10,1
1959-10-14,10,14,1959,1959.0,47.0000,1959.10,1
...,...,...,...,...,...,...,...
2000-07-27,7,27,2000,1999.0,109.9375,2000.07,10
2000-07-28,7,28,2000,1999.0,107.7500,2000.07,10
2000-07-29,7,29,2000,1999.0,105.5625,2000.07,10
